<a href="https://colab.research.google.com/github/Surajpatra700/YoloV8_CV/blob/main/candies_track%26count_numbers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [40]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
# %cd {HOME}
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-' -O- | sed -rn 's/.confirm=([0-9A-Za-z_]+)./\1\n/p')&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-" -O 70899749.mp4 && rm -rf /tmp/cookies.txt

/content
--2023-12-07 14:16:54--  https://docs.google.com/uc?export=download&confirm=&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-
Resolving docs.google.com (docs.google.com)... 64.233.183.100, 64.233.183.102, 64.233.183.139, ...
Connecting to docs.google.com (docs.google.com)|64.233.183.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-ag-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/k4ej4n5qtbg9b5l5tv9t5fa27dkv1e1v/1701958575000/04309230031174164349/*/1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-?e=download&uuid=1043b1e2-1116-4a9b-a70c-3af80188a15a [following]
--2023-12-07 14:17:11--  https://doc-0s-ag-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/k4ej4n5qtbg9b5l5tv9t5fa27dkv1e1v/1701958575000/04309230031174164349/*/1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-?e=download&uuid=1043b1e2-1116-4a9b-a70c-3af80188a15a
Resolving doc-0s-ag-docs.googleusercontent.com (doc-0s-ag-docs.googleusercontent.com)... 142.250.

In [ ]:
%cd {HOME}
SOURCE_VIDEO_PATH = f"{HOME}/70899749.mp4"

In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.223 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.0/78.2 GB disk)


In [ ]:
%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git

# %cd {HOME}/ByteTrack

# # workaround related to https://github.com/roboflow/notebooks/issues/80
# !sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

!cd ByteTrack && pip3 install -q -r requirements.txt
!cd ByteTrack && python3 setup.py -q develop
!pip install -q cython_bbox
!pip install -q onemetric
!pip install -q loguru lap thop

from IPython import display
display.clear_output()


import sys
sys.path.append(f"{HOME}/ByteTrack")


import yolox
print("yolox._version:", yolox.__version__)

yolox._version: 0.1.0


In [ ]:
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass

@dataclass(frozen=True)
class BYTETrackerArgs:
  track_thresh: float = 0.25
  track_buffer: int = 30
  match_thresh: float = 0.8
  aspect_ratio_thresh: float = 3.0
  min_box_area: float = 1.0
  mot20: bool = False

In [ ]:
!pip install -i https://test/pypi.org/simple/ supervision


from IPython import display
display.clear_output()


import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.1.0


In [ ]:
from typing import List
from supervision.tools.detections import Detections, BoxAnnotator

import numpy as np


# converts Detections into format that can be consumed by match_detections_with_tracks function
def detections2boxes(detections: Detections) -> np.ndarray:
    return np.hstack((
        detections.xyxy,
        detections.confidence[:, np.newaxis]
    ))


# converts List[STrack] into format that can be consumed by match_detections_with_tracks function
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)


# matches our bounding boxes with predictions
def match_detections_with_tracks(
    detections: Detections,
    tracks: List[STrack]
) -> Detections:
    if not np.any(detections.xyxy) or len(tracks) == 0:
        return np.empty((0,))

    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detections.xyxy)
    track2detection = np.argmax(iou, axis=1)

    tracker_ids = [None] * len(detections)

    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            tracker_ids[detection_index] = tracks[tracker_index].track_id

    return tracker_ids

In [ ]:
%cd {HOME}
!unzip weights.zip

/content
Archive:  weights.zip
replace runs/detect/train/weights/best.pt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: runs/detect/train/weights/best.pt  
replace runs/detect/train/weights/last.pt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: runs/detect/train/weights/last.pt  


In [ ]:
# settings
MODEL = "runs/detect/train/weights/best.pt"

In [ ]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


In [ ]:
# dict mapping class_id to class_name
CLASS_NAMES_DICT = model.model.names

# class_ids of interest - candies
CLASS_ID = [0]

In [ ]:
TARGET_VIDEO_PATH = f"{HOME}/candies-count.mp4"

In [ ]:
from supervision.video.source import get_video_frames_generator

# importing utility for displaying the picked frame in notebook
from supervision.notebook.utils import show_frame_in_notebook
from supervision.tools.detections import Detections, BoxAnnotator
from supervision.draw.color import ColorPalette
from supervision.video.sink import VideoSink
from supervision.tools.line_counter import LineCounter, LineCounterAnnotator
from supervision.geometry.dataclasses import Point

from supervision.video.dataclasses import VideoInfo

from tqdm.notebook import tqdm

byte_tracker = BYTETracker(BYTETrackerArgs())

# frame generator used to read frames one after another
generator = get_video_frames_generator(SOURCE_VIDEO_PATH)

line_counter = LineCounter(start=Point(50, 50), end=Point(3840-50, 2160-50))

box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=2)

# line annotator for displaying the line
line_annotator = LineCounterAnnotator(thickness=4, text_thickness = 4, text_scale=2)

video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

with VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
  for frame in tqdm(generator, total=video_info.total_frames):

    # picking a frame from the generator
    # iterator = iter(generator)

    # # picking the next frame
    # frame = next(iterator)

    results = model(frame)[0]

    detections = Detections(
        xyxy = results.boxes.xyxy.cpu().numpy(),
        confidence = results.boxes.conf.cpu().numpy(),
        class_id = results.boxes.cls.cpu().numpy().astype(int)
    )

    tracks = byte_tracker.update(
        output_results = detections2boxes(detections=detections),
        img_info = frame.shape,
        img_size = frame.shape
    )
    tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
    detections.tracker_id = np.array(tracker_id)

    labels = [
        f"#{tracker_id} {CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
        for _, confidence, class_id, tracker_id in detections
    ]

    line_counter.update(detections=detections)

    frame = box_annotator.annotate(frame=frame, detections=detections,labels=labels)
    line_annotator.annotate(frame=frame, line_counter=line_counter)

    sink.write_frame(frame)
    # show_frame_in_notebook(frame, (16,16))   #--> This line display the image in notebook